In [2]:
#Initialization for HateSpeechRecognition
import scipy as sp
import numpy as np
import pandas as pd
import sklearn
from sklearn import svm
import nltk
import re
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import zero_one_loss
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from scipy.stats import t
from nltk.probability import FreqDist
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/mufa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mufa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mufa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
def nested_kfold_cv(clf, param_grid, X, y, k1=10, k2=3):
    
    err = []
    kfold = KFold(n_splits=k1, shuffle=True, random_state=42)
    
    # Outer loop
    for ind_train, ind_test in kfold.split(X):
        
        X_train, y_train, X_test, y_test = X[ind_train], y[ind_train], X[ind_test], y[ind_test]
        
        # Inner loop
        inn = GridSearchCV(clf,param_grid,n_jobs=5, cv=StratifiedKFold(n_splits=5, 
                                              random_state=42).split(X_train, y_train), 
                           verbose=2).fit(X_train, y_train)
        
        # Prediction based on the best selected params, the ones that minimize average error
        h = inn.best_estimator_.fit(X_train, y_train).predict(X_test)
        
        err.append(zero_one_loss(y_test, h))
        print(classification_report( y_test, h ))
    return err

In [4]:

def clean_text(rgx_list, text):
    new_text = text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, '', new_text)
    return new_text

In [5]:
def get_common_words(content):
    ret_list = []
    fdist2 = nltk.FreqDist(content)
    most_list = fdist2.most_common(75)
    for x in most_list:
        ret_list.append(x)
    return ret_list

In [6]:

stopwords_list = stopwords.words('english')
#other_exclusions = ["#ff", "ff", "rt","!",":","...",".","-","&","?"]
#stopwords_list.extend(other_exclusions)

dataset = pd.read_csv("dataset/labeled_data.csv")

tweets = dataset.tweet

ps = nltk.stem.PorterStemmer()

#ps.stem(dataset)

#Text cleaning and tokenization, then stemming then POS tagging
filtered_tweets=[];
tweet_tags = [];
filtered_tweets_stemmed=[];
common_words_prepare=[];

for tweet in tweets:
    tweet = clean_text(["http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",".*@.*:","&#*\w*","@[\w\-]+","[^\w\s]"],tweet)
    tweet = tweet.lower()
    word_tokens = word_tokenize(tweet)
    common_words = get_common_words(word_tokens)
    for word in common_words:
        common_words_prepare.append(word)
    filtered_tweets.append([word for word in word_tokens if not word in stopwords_list])
    filtered_tweets_stemmed.append([ps.stem(word) for word in word_tokens if not word in stopwords_list])
    
for tweet in filtered_tweets:
    tweet_tags.append(nltk.pos_tag(tweet))
    


In [7]:
# Izbacivanje frequent wordsa

common_words = get_common_words(common_words_prepare)
#for word in filtered_tweets[1:100]:
#    print(word)
common = []
for word in common_words[1:50]:
   common.append(word[0][0])

filtered_tweets_no_common = [];
filtered_tweets_no_common_stemmed = [];
for line in filtered_tweets:
    filtered_tweets_no_common.append([word for word in line if not word in common])
    filtered_tweets_no_common_stemmed.append([ps.stem(word) for word in line if not word in common])
#print("---------------------------")
#for word in filtered_tweets_no_common[1:100]:
#    print(word)
    

In [8]:
#Creating TfIdf vectorizer
ftss=[]
for tweet in filtered_tweets_no_common_stemmed:
    ftss.append(' '.join(tweet))

vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,3),lowercase=False,max_features=1000,smooth_idf=False,norm=None,max_df=0.75,min_df=5)



In [9]:
X = vectorizer.fit_transform(ftss).toarray()
Y = dataset['class']

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.3)



In [15]:
#SVM Classifier
C = [ 2e-2, 2e-1, 2e-0]
gama = [ 2e-2, 2e-1, 2e-0]
param = [{'svc__kernel': ['rbf'], 'svc__C': C}]
clf = svm.SVC(decision_function_shape='ovo')
scale = StandardScaler()
pipeline = Pipeline([('scaler', scale), ('svc', clf)])

svm_err = nested_kfold_cv(pipeline, param, X, Y)
#param_grid = [{}] 
#grid_search = GridSearchCV(pipeline, 
#                           param_grid,
#                          n_jobs = 5,
#                           cv=StratifiedKFold(n_splits=5, 
#                                              random_state=42).split(X_train, y_train), 
#                           verbose=2)
#model = grid_search.fit(X_train,y_train)
#y_pred = model.predict(X_test)
#report = classification_report( y_test, y_pred )
#print(report)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed: 69.7min finished


             precision    recall  f1-score   support

          0       0.48      0.12      0.19       164
          1       0.84      0.97      0.90      1905
          2       0.79      0.44      0.56       410

avg / total       0.80      0.82      0.80      2479

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed: 69.8min finished


             precision    recall  f1-score   support

          0       0.38      0.16      0.22       127
          1       0.84      0.97      0.90      1927
          2       0.82      0.42      0.56       425

avg / total       0.82      0.83      0.81      2479

Fitting 5 folds for each of 3 candidates, totalling 15 fits


...

             precision    recall  f1-score   support

          0       0.45      0.13      0.21       135
          1       0.84      0.97      0.90      1907
          2       0.82      0.45      0.58       436

avg / total       0.82      0.83      0.81      2478

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed: 72.9min finished


             precision    recall  f1-score   support

          0       0.63      0.13      0.21       133
          1       0.86      0.97      0.91      1937
          2       0.81      0.49      0.61       408

avg / total       0.84      0.85      0.82      2478

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed: 68.5min finished


             precision    recall  f1-score   support

          0       0.42      0.13      0.20       124
          1       0.86      0.98      0.91      1942
          2       0.86      0.50      0.63       412

avg / total       0.84      0.85      0.83      2478



In [14]:
#Logistic Regression

C = [2e-3, 2e-2, 2e-1, 2e-0, 2e-1, 2e-2, 2e-3]

std_scaler = StandardScaler()
clf = LogisticRegression(multi_class='ovr',solver='newton-cg')
pipeline = Pipeline([('scaler', std_scaler)  , ('clf', clf)])
param = [{'clf__C': C}]

logreg_err = nested_kfold_cv(pipeline, param, X, Y)



#TODO: logicstic regression, model selection, kfold

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=5)]: Done  35 out of  35 | elapsed:  5.9min finished


             precision    recall  f1-score   support

          0       0.53      0.10      0.17       164
          1       0.85      0.97      0.90      1905
          2       0.80      0.54      0.65       410

avg / total       0.82      0.84      0.81      2479

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=5)]: Done  35 out of  35 | elapsed:  5.9min finished


             precision    recall  f1-score   support

          0       0.47      0.12      0.19       127
          1       0.86      0.97      0.91      1927
          2       0.80      0.50      0.62       425

avg / total       0.83      0.85      0.82      2479

Fitting 5 folds for each of 7 candidates, totalling 35 fits


...

             precision    recall  f1-score   support

          0       0.41      0.11      0.17       135
          1       0.85      0.96      0.90      1907
          2       0.80      0.55      0.65       436

avg / total       0.82      0.84      0.82      2478

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=5)]: Done  35 out of  35 | elapsed:  5.9min finished


             precision    recall  f1-score   support

          0       0.52      0.11      0.17       133
          1       0.86      0.97      0.91      1937
          2       0.79      0.55      0.65       408

avg / total       0.83      0.85      0.83      2478

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=5)]: Done  35 out of  35 | elapsed:  5.9min finished


             precision    recall  f1-score   support

          0       0.57      0.17      0.26       124
          1       0.88      0.98      0.92      1942
          2       0.86      0.58      0.70       412

avg / total       0.86      0.87      0.85      2478



In [16]:
from scipy.stats import ttest_rel as paired_t_test

t_stat, p_val = paired_t_test(logreg_err, svm_err)

if p_val <= 0.05:
    print('\np-value = ', p_val, ', hipoteza se odbacuje.')
else:
    print('\np-value = ', p_val, ', hipoteza se ne odbacuje.')


p-value =  5.310675075060413e-05 , hipoteza se odbacuje.


In [10]:
#Sampling data so we get equal distribution for all classes. used in active learning
d0 = dataset[dataset['class'] == 0]
d1 = dataset[dataset['class'] == 1].sample(d0.shape[0])
d2 = dataset[dataset['class'] == 2].sample(d0.shape[0])

#Dl data that is representing same number of all classes. From this we will take n data for starter training as
#labeled data, and rest for unlabeled data for active learning
Dl = pd.concat([d0,d1,d2])
Dl = sklearn.utils.shuffle(Dl)
Dl.reset_index(drop=True)





,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,21500,3,0,3,0,1,"Some of yall just ""popular"" cuz of yall name ...."
1,4531,3,0,0,3,2,@Pepper_Redbone @Yankees @Mets Exactly by folk...
2,19341,3,0,0,3,2,"RT @dril: night time falls. im ""corie latin"" n..."
3,12381,3,0,0,3,2,Jihadis taunting America &amp; spreading paran...
4,18453,3,0,0,3,2,RT @WhatTheFFacts: Actor Charlie Sheen once bo...
5,11303,3,0,0,3,2,I want Oreos!! Who can help me
6,2342,3,2,1,0,0,@05proFESSOR I bet he's a Starbucks queer
7,15968,3,0,3,0,1,RT @Ih8lightskins__: only a black man can make...
8,6599,3,2,1,0,0,@kieffer_jason @zach_smith98 @NathannDevlin so...
9,21524,3,2,0,1,0,Someone shut this beaner up please.


In [11]:
AL_filtered_tweets=[];
AL_tweet_tags = [];
AL_filtered_tweets_stemmed=[];
AL_common_words_prepare=[];

for tweet in Dl['tweet']:
    tweet = clean_text(["http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",".*@.*:","&#*\w*","@[\w\-]+","[^\w\s]"],tweet)
    tweet = tweet.lower()
    word_tokens = word_tokenize(tweet)
    common_words = get_common_words(word_tokens)
    for word in common_words:
        AL_common_words_prepare.append(word)
    AL_filtered_tweets.append([word for word in word_tokens if not word in stopwords_list])
    AL_filtered_tweets_stemmed.append([ps.stem(word) for word in word_tokens if not word in stopwords_list])
    
for tweet in AL_filtered_tweets:
    AL_tweet_tags.append(nltk.pos_tag(tweet))

# Izbacivanje frequent wordsa

AL_common_words = get_common_words(AL_common_words_prepare)
#for word in filtered_tweets[1:100]:
#    print(word)
AL_common = []
for word in AL_common_words[1:50]:
   AL_common.append(word[0][0])

AL_filtered_tweets_no_common = [];
AL_filtered_tweets_no_common_stemmed = [];
for line in AL_filtered_tweets:
    AL_filtered_tweets_no_common.append([word for word in line if not word in AL_common])
    AL_filtered_tweets_no_common_stemmed.append([ps.stem(word) for word in line if not word in AL_common])
#print("---------------------------")
#for word in filtered_tweets_no_common[1:100]:
#    print(word)

AL_ftss=[]
for tweet in AL_filtered_tweets_no_common_stemmed:
    AL_ftss.append(' '.join(tweet))


In [90]:
AL_X = vectorizer.fit_transform(AL_ftss).toarray()
AL_Y = Dl['class'].tolist()

In [91]:
import math
Tk_X, Tu_X, Tk_y, _ = train_test_split(AL_X, AL_Y, random_state=42, test_size=0.8)

n_labeled_points = math.ceil(len(AL_Y)*0.05)
unlabeled_indices = np.arange(len(AL_Y))[n_labeled_points:]


In [92]:
from sklearn.semi_supervised import label_propagation
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse.csgraph import connected_components
import matplotlib.pyplot as plt
from scipy import stats
max_iterations = 100
f = plt.figure()
for i in range(max_iterations):
    if len(Tu) == 0 :
        print("No unlabeled data to label.")
        break
    al_model = label_propagation.LabelSpreading(gamma=0.25,max_iter=50)
    y_train = np.copy(AL_Y)
    y_train[unlabeled_indices] = -1
    al_model.fit(AL_X,y_train)
    predicted_labels = al_model.transduction_[unlabeled_indices]
    true_labels = [AL_Y[i] for i in unlabeled_indices]
    if set(al_model.classes_).issubset(set(true_labels)) == False:
        break
    cm = confusion_matrix(true_labels, predicted_labels,
                          labels=al_model.classes_)
    print("Iteration %i %s" % (i, 70 * "_"))
    
        
    print("Label Spreading model: %d labeled & %d unlabeled (%d total)"
          % (n_labeled_points, len(AL_Y) - n_labeled_points,
             len(AL_Y)))

    print(classification_report(true_labels, predicted_labels))

    print("Confusion matrix")
    print(cm)

    # compute the entropies of transduced label distributions
    pred_entropies = stats.distributions.entropy(
        al_model.label_distributions_.T)

    # select up to 5 digit examples that the classifier is most uncertain about
    uncertainty_index = np.argsort(pred_entropies)[::-1]
    uncertainty_index = uncertainty_index[
        np.in1d(uncertainty_index, unlabeled_indices)][:50]

    # keep track of indices that we get labels for
    delete_indices = np.array([])
    
    for index, check_index in enumerate(uncertainty_index):
        check = AL_ftss[check_index]

        delete_index, = np.where(unlabeled_indices == check_index)
        delete_indices = np.concatenate((delete_indices, delete_index))

    unlabeled_indices = np.delete(unlabeled_indices, delete_indices)
    n_labeled_points += len(uncertainty_index)



Iteration 0 ______________________________________________________________________
Label Spreading model: 215 labeled & 4075 unlabeled (4290 total)
             precision    recall  f1-score   support

          0       0.51      0.33      0.40      1353
          1       0.38      0.68      0.49      1371
          2       0.53      0.29      0.37      1351

avg / total       0.47      0.43      0.42      4075

Confusion matrix
[[449 746 158]
 [248 931 192]
 [180 779 392]]


/home/mufa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


Iteration 1 ______________________________________________________________________
Label Spreading model: 265 labeled & 4025 unlabeled (4290 total)
             precision    recall  f1-score   support

          0       0.51      0.36      0.42      1332
          1       0.39      0.65      0.48      1359
          2       0.52      0.32      0.39      1334

avg / total       0.47      0.44      0.43      4025

Confusion matrix
[[475 682 175]
 [259 883 217]
 [191 719 424]]
Iteration 2 ______________________________________________________________________
Label Spreading model: 315 labeled & 3975 unlabeled (4290 total)
             precision    recall  f1-score   support

          0       0.51      0.36      0.42      1309
          1       0.39      0.66      0.49      1343
          2       0.53      0.31      0.39      1323

avg / total       0.48      0.44      0.43      3975

Confusion matrix
[[468 680 161]
 [252 886 205]
 [189 724 410]]
Iteration 3 ______________________________

Iteration 19 ______________________________________________________________________
Label Spreading model: 1165 labeled & 3125 unlabeled (4290 total)
             precision    recall  f1-score   support

          0       0.53      0.40      0.46      1075
          1       0.40      0.58      0.48      1015
          2       0.55      0.46      0.50      1035

avg / total       0.50      0.48      0.48      3125

Confusion matrix
[[429 464 182]
 [221 591 203]
 [153 409 473]]
Iteration 20 ______________________________________________________________________
Label Spreading model: 1215 labeled & 3075 unlabeled (4290 total)
             precision    recall  f1-score   support

          0       0.53      0.40      0.45      1061
          1       0.41      0.57      0.48      1000
          2       0.54      0.48      0.51      1014

avg / total       0.50      0.48      0.48      3075

Confusion matrix
[[422 444 195]
 [220 569 211]
 [152 379 483]]
Iteration 21 _________________________

Iteration 37 ______________________________________________________________________
Label Spreading model: 2065 labeled & 2225 unlabeled (4290 total)
             precision    recall  f1-score   support

          0       0.55      0.41      0.47       788
          1       0.42      0.54      0.47       700
          2       0.57      0.58      0.57       737

avg / total       0.52      0.51      0.51      2225

Confusion matrix
[[323 304 161]
 [167 375 158]
 [ 93 218 426]]
Iteration 38 ______________________________________________________________________
Label Spreading model: 2115 labeled & 2175 unlabeled (4290 total)
             precision    recall  f1-score   support

          0       0.55      0.41      0.47       765
          1       0.42      0.53      0.46       685
          2       0.57      0.58      0.58       725

avg / total       0.51      0.50      0.50      2175

Confusion matrix
[[316 293 156]
 [170 360 155]
 [ 93 213 419]]
Iteration 39 _________________________

Iteration 55 ______________________________________________________________________
Label Spreading model: 2965 labeled & 1325 unlabeled (4290 total)
             precision    recall  f1-score   support

          0       0.53      0.41      0.46       424
          1       0.44      0.60      0.51       402
          2       0.67      0.62      0.64       499

avg / total       0.56      0.54      0.54      1325

Confusion matrix
[[172 170  82]
 [ 93 241  68]
 [ 60 131 308]]
Iteration 56 ______________________________________________________________________
Label Spreading model: 3015 labeled & 1275 unlabeled (4290 total)
             precision    recall  f1-score   support

          0       0.53      0.41      0.46       409
          1       0.43      0.59      0.50       379
          2       0.67      0.62      0.65       487

avg / total       0.56      0.54      0.54      1275

Confusion matrix
[[167 163  79]
 [ 90 223  66]
 [ 59 127 301]]
Iteration 57 _________________________

Iteration 73 ______________________________________________________________________
Label Spreading model: 3865 labeled & 425 unlabeled (4290 total)
             precision    recall  f1-score   support

          0       0.53      0.42      0.47       120
          1       0.50      0.61      0.55       115
          2       0.78      0.77      0.77       190

avg / total       0.63      0.63      0.63       425

Confusion matrix
[[ 51  43  26]
 [ 29  70  16]
 [ 16  28 146]]
Iteration 74 ______________________________________________________________________
Label Spreading model: 3915 labeled & 375 unlabeled (4290 total)
             precision    recall  f1-score   support

          0       0.56      0.46      0.51       102
          1       0.51      0.61      0.56       101
          2       0.80      0.79      0.80       172

avg / total       0.66      0.65      0.65       375

Confusion matrix
[[ 47  36  19]
 [ 24  62  15]
 [ 13  23 136]]
Iteration 75 ___________________________

In [93]:
f.suptitle("Active learning with Label Propagation.\nRows show 50 most "
           "uncertain labels to learn with the next model.", y=1.15)
plt.subplots_adjust(left=0.2, bottom=0.03, right=0.9, top=0.9, wspace=0.2,
                    hspace=0.85)
plt.show()